## Reduction operation: the sum of the numbers in the range [0, value)

In [3]:
import numpy as np
import sys

def reduc_operation(A):
    """Compute the sum of the elements of Array A in the range [0, value)."""
    s = 0
    for i in range(A.size):
        s += A[i]
    return s

# Secuencial

value = int(sys.argv[1])

X = np.random.rand(value)

# Para imprimir los pimeros valores del array

# print(X[0:12])

# Utilizando las operaciones mágicas de ipython

tiempo = %timeit -r 2 -o -q reduc_operation(X)

print("Time taken by reduction operation using a function:", tiempo)


print(f"And the result of the sum of numbers in the range [0, value) is: {reduc_operation(X)}\n")


# Utilizando numpy.sum()

tiempo = %timeit -r 2 -o -q np.sum(X)

print("Time taken by reduction operation using numpy.sum():", tiempo)

print("Now, the result using numpy.sum():", np.sum(X),"\n ")


# Utilizando numpy.ndarray.sum()

tiempo= %timeit -r 2 -o -q X.sum()

print("Time taken by reduction operation using numpy.ndarray.sum():", tiempo)

print("Now, the result using numpy.ndarray.sum():", X.sum())

Time taken by reduction operation using a function: 5.3 ms ± 134 µs per loop (mean ± std. dev. of 2 runs, 100 loops each)
And the result of the sum of numbers in the range [0, value) is: 24989.504831953385

Time taken by reduction operation using numpy.sum(): 13.8 µs ± 9.58 ns per loop (mean ± std. dev. of 2 runs, 100,000 loops each)
Now, the result using numpy.sum(): 24989.504831953527 
 
Time taken by reduction operation using numpy.ndarray.sum(): 11.7 µs ± 5.23 ns per loop (mean ± std. dev. of 2 runs, 100,000 loops each)
Now, the result using numpy.ndarray.sum(): 24989.504831953527


### 3.3. Python HPC: Paralelismo con GPUs

#### EJERCICIO A

En esta parte del laboratorio vamos a usar paralelismo con GPUs como si fuera un acelerador de código para reducir el tiempo de ejecución de programas escritos en Python.
Tenemos que modificar el código y adaptarlo con el paquete cupy, que es una librería similar a NumPy.

In [24]:
# Importamos primeramente la librería CuPy y las librerías para medir los tiempos:
import cupy as cp # Esta librería es similar a Numpy pero en lugar de CPU, usa la GPU para acelerar los cálculos.
import time
from cupyx.profiler import benchmark

# Creamos el array adaptado a cupy
X_gpu = cp.random.rand(value)

# Medimos el tiempo de ejecución de la suma con time 
start = time.time()
result = cp.sum(X_gpu)
cp.cuda.Device().synchronize()  # Sincronizamos la GPU
end = time.time()

print(f"Tiempo que tarda la operación de reducción usando CuPy: {end - start:.6f} segundos")

# Alternativa
# Función a medir: suma en la GPU. Se hace este paso porque la función que vamos a medir debe estar definida explícitamente
def sum_gpu(arr):
    return cp.sum(arr)

# Usamos benchmark para medir el tiempo de ejecución
execution_gpu = benchmark(sum_gpu, (X_gpu,), n_repeat=10, n_warmup=2)
gpu_avg_time = np.average(execution_gpu.gpu_times)*1e3

print(f"Tiempo promedio que tarda la operación de reducción usando CuPy (benchmark): {gpu_avg_time:.6f} ms")
print("\n")
print(f"Resultado de la suma: {sum_gpu(X_gpu)}")

Tiempo que tarda la operación de reducción usando CuPy: 0.000522 segundos
Resultado de la suma: 25057.379951513616
Tiempo promedio que tarda la operación de reducción usando CuPy (benchmark): 0.036918 ms
Resultado de la suma: 25057.379951513616


Para medir correctamente el tiempo en operaciones con GPU, es mejor que usemos time.time() en lugar de %timeit, porque las tareas en la GPU son asíncronas y Python no espera a que terminen antes de seguir ejecutando el código. Esto significa que %timeit podría estar midiendo solo el tiempo de envío de las tareas, no su ejecución completa. En cambio, con time.time(), podemos incluir una sincronización explícita (cp.cuda.Device().synchronize() en CuPy o cuda.synchronize() en Numba) para asegurarnos de que todas las operaciones en la GPU hayan terminado antes de calcular el tiempo, lo que garantiza que las mediciones sean precisas.

Otra alternativa sería usar cupyx.profiler.benchmark(), que ya está pensado para medir el tiempo de las operaciones en la GPU de forma más precisa. A diferencia de time.time(), este método hace varias repeticiones (y hasta calentamientos previos) para que las mediciones sean más confiables. Además, incluye sincronizaciones automáticamente, así que no tienes que preocuparte por eso. Es súper útil cuando quieres medir el tiempo exacto que toma una operación en la GPU, sin andar haciendo sincronizaciones manuales.

La diferencia en los tiempos puede deberse a que el tiempo total (0.000522 s) incluye tareas adicionales como la configuración y sincronización de la GPU, mientras que el tiempo promedio del benchmark (0.036918 ms) mide exclusivamente el tiempo de ejecución interna de la operación de reducción en la GPU, sin estos pasos adicionales.

#### EJERCICIO B

Cuando trabajamos con GPUs, Numba nos facilita la optimización utilizado unas funciones de alto nivel llamadas ufuncs (o funciones universales). Estas se ejecutan de forma paralela en la GPU. 
- El decorador @vectorize convierte una función Python en una ufunc.
- Cuando usamos target='cuda', indicamos que esta ufunc debe ejecutarse en la GPU.

Para trabajar con GPU, los datos deben transferirse desde la memoria de la CPU a la memoria de la GPU. Esto se realiza con la función cuda.to_device de Numba. Después de completar las operaciones en la GPU, los datos se transfieren de regreso a la CPU si es necesario.

Las operaciones en la GPU son asíncronas, lo que significa que Python continúa ejecutando el código mientras la GPU procesa los datos. Para garantizar que los cálculos en la GPU hayan terminado antes de medir el tiempo o usar los resultados, se utiliza la función cuda.synchronize.

Según el profesor: debido a una mayor complicación de lo esperado, se anula este apartado.

#### EJERCICIO C

- Ejecutando la operación de reducción con 5000000 elementos

Time taken by reduction operation using a function: 263 ms ± 475 µs per loop (mean ± std. dev. of 2 runs, 1 loop each)
And the result of the sum of numbers in the range [0, value) is: 2500716.024539858

Time taken by reduction operation using numpy.sum(): 1.38 ms ± 1.08 µs per loop (mean ± std. dev. of 2 runs, 1,000 loops each)
Now, the result using numpy.sum(): 2500716.024540222

Time taken by reduction operation using numpy.ndarray.sum(): 1.39 ms ± 5.12 µs per loop (mean ± std. dev. of 2 runs, 1,000 loops each)
Now, the result using numpy.ndarray.sum(): 2500716.024540222

Tiempo que tarda la operación de reducción usando CuPy: 0.380770 segundos
Tiempo promedio que tarda la operación de reducción usando CuPy (benchmark): 0.085398 ms

Resultado de la suma: 2500455.3354769163

- Ejecutando la operación de reducción con 50000000 elementos

Time taken by reduction operation using a function: 2.61 s ± 1.18 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)
And the result of the sum of numbers in the range [0, value) is: 25000019.474952582

Time taken by reduction operation using numpy.sum(): 17.6 ms ± 548 ns per loop (mean ± std. dev. of 2 runs, 100 loops each)
Now, the result using numpy.sum(): 25000019.474950016

Time taken by reduction operation using numpy.ndarray.sum(): 17.6 ms ± 9.98 µs per loop (mean ± std. dev. of 2 runs, 100 loops each)
Now, the result using numpy.ndarray.sum(): 25000019.474950016

Tiempo que tarda la operación de reducción usando CuPy: 0.019101 segundos
Tiempo promedio que tarda la operación de reducción usando CuPy (benchmark): 0.702125 ms

Resultado de la suma: 24994568.087425876

- Ejecutando la operación de reducción con 500000000 elementos

Time taken by reduction operation using a function: 52.8 s ± 145 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)
And the result of the sum of numbers in the range [0, value) is: 250000017.95264807

Time taken by reduction operation using numpy.sum(): 216 ms ± 78.6 µs per loop (mean ± std. dev. of 2 runs, 1 loop each)
Now, the result using numpy.sum(): 250000017.9526514

Time taken by reduction operation using numpy.ndarray.sum(): 216 ms ± 103 µs per loop (mean ± std. dev. of 2 runs, 1 loop each)
Now, the result using numpy.ndarray.sum(): 250000017.9526514

Tiempo que tarda la operación de reducción usando CuPy: 0.040583 segundos
Tiempo promedio que tarda la operación de reducción usando CuPy (benchmark): 6.850957 ms

Resultado de la suma: 249996452.15224293


#### EJERCICIO D

Al utilizar los paquetes CuPy y Numba, observamos una mejora significativa en los tiempos de ejecución al realizar operaciones de reducción en comparación con enfoques secuenciales o con NumPy en CPU. Esta mejora se debe a que ambos paquetes aprovechan la capacidad paralela de la GPU, que está diseñada para procesar grandes volúmenes de datos de forma simultánea.

En el caso de CuPy, el tiempo interno de la operación (según el benchmark) es extremadamente bajo, especialmente para arrays grandes, lo que refleja la eficiencia del cálculo directo en la GPU. Sin embargo, los tiempos totales incluyen sobrecargas como la transferencia de datos entre la CPU y la GPU, lo que puede hacer que los tiempos sean más cercanos a los de NumPy cuando el tamaño del array es pequeño. A medida que aumentamos el número de elementos, la ventaja de la GPU se vuelve más evidente, ya que su arquitectura paralela maneja mucho mejor los datasets grandes.

Numba, por su parte, muestra un comportamiento similar, ya que también utiliza la GPU para optimizar las operaciones. Sin embargo, la configuración inicial y las transferencias de datos tienen un impacto notable en los tiempos totales, especialmente para tamaños de arrays pequeños. Esto sugiere que tanto CuPy como Numba son herramientas ideales cuando trabajamos con grandes volúmenes de datos y operaciones complejas, mientras que para arrays más pequeños, NumPy sigue siendo competitivo debido a su menor sobrecarga inicial.

En resumen, los resultados reflejan que tanto CuPy como Numba son opciones eficientes para operaciones paralelas en GPU, y su ventaja crece conforme se incrementa el tamaño de los datos.